In [1]:
import bento
from bento.client import Client
from bento.sim import Simulation
from bento.example.mountcar import MountainCar, Action, State
from bento.example.specs import Velocity, Position

In [2]:
ENGINE_HOST = 'bento.mrzzy.co'
ENGINE_PORT = '54242'

In [3]:
client = Client(host=ENGINE_HOST, port=ENGINE_PORT)

In [4]:
client.connect(timeout_sec=30)

True

In [6]:
client.remove_sim('mountain_car')

In [7]:
sim = Simulation.from_def(MountainCar, client)

In [9]:
sim.start()

In [16]:
env = sim.entity(components=[Action, State])
car = sim.entity(components=[Velocity, Position])

In [22]:
car[Position].x

-0.5376745380194506

In [21]:
sim.step()

In [23]:
env[State].reward

-1

In [28]:
from v2.env import Env
from bento.client import Client
from bento.sim import Simulation
from bento.example.mountcar import MountainCar, Action, State
from bento.example.specs import Velocity, Position
import gym

class BentoMountainCarEnv(Env):
    def __init__(self, host=ENGINE_HOST, port=ENGINE_PORT):
        # Bento
        client = Client(host=host, port=port)
        status = client.connect(timeout_sec=30)
        print('Client Status:', status)
        self.client = client
        self.sim = None
        self.reset()
        # Gym
        env = gym.make('MountainCar-v0').unwrapped
    def rand_state(self):
        return self.env.observation_space.sample()
    def rand_action(self):
        return self.env.action_space.sample()
    def reset(self):
        self.client.remove_sim(MountainCar)
        self.sim = Simulation.from_def(MountainCar, self.client)
        self.sim.start()
        state = self.get_state()
        self.env.state = state
        return state
    def step(self, action):
        self.move_car(self, action)
        state, reward, done = self.get_state(), self.get_reward(), self.get_done()
        self.env.state = state
        return state, reward, done
    def render(self):
        self.env.render()
    ## Bento Helper
    def get_entities(self):
        car = self.sim.entity(components=[Velocity, Position])
        env = self.sim.entity(components=[Action, State])
    def get_state(self):
        car, env = self.get_entities()
        return np.array([car[Position].x, car[Velocity].x])
    def get_reward(self):
        car, env = self.get_entities()
        return env[State].reward
    def get_done(self):
        car, env = self.get_entities()
        return env[State].ended
    def move_car(self, action):
        car = self.get_car()
        car[Action].accelerate = action
        self.sim.step()

ModuleNotFoundError: No module named 'v2'

In [24]:
import gym

In [25]:
env = gym.make('MountainCar-v0')

In [26]:
env.observation_space

Box(-1.2000000476837158, 0.6000000238418579, (2,), float32)

In [27]:
env.observation_space.sample()

array([0.1393554 , 0.06834549], dtype=float32)

In [4]:
MountainCar.name

'mountain_car'